# Identify Objects in Observed Image
Written by Kiyoaki Okudaira<br>
*Kyushu University Hanada Lab / University of Washington / IAU CPS SatHub<br>
(okudaira.kiyoaki.528@s.kyushu-u.ac.jp or kiyoaki@uw.edu)<br>
<br>
Identify satellites and debris in observed FITS image<br>
<br>
**History**<br>
coding 2025-12-11 : 1st coding<br>
<br>
(c) 2025 Kiyoaki Okudaira - Kyushu University Hanada Lab (SSDL) / University of Washington / IAU CPS SatHub

### Parameters
**Image PATH**

In [ ]:
image_PATH = "/Users/kiyoaki/VScode/satphotometry_package/output/images/Capture_00003.fits" # FITS file PATH | str
search_range = 2 # Object search range [img] | int

**Observation**<br>
All datetime should be in UTC

In [ ]:
obs_metadata = "AUTO" # "AUTO" or "MANUAL" : If "AUTO", observation date and exposure read from header | str
obs_begin    = "2025-12-11T10:00:00.000" # UTC Date and time observation begin [YYYY-MM-DDTHH:MM:SS.SSS] | str
obs_exposure = 1 # Exposure [sec] | int or float

**TLE Download settings**<br>
Downloading from space-track.org

In [ ]:
tle_dt_range = 1 # TLE range from observation date | int > 0

### Import and initial settings

**PATH setting**

In [ ]:
base_PATH = "/Users/kiyoaki/VScode/satphotometry_package/"
output_PATH = base_PATH + "output"
input_PATH = base_PATH + "input"
spice_myfile_PATH = "/Users/kiyoaki/VScode/satphotometry_package/config/myfile.txt"
tle_PATH = "/Users/kiyoaki/VScode/satphotometry_package/input/tle/13056.txt" # If download_tle is False | str

**SPICE toolkit**<br>
import spiceypy and start up SPICE kernel

In [ ]:
import spiceypy as spice
spice.furnsh(spice_myfile_PATH)

**Standard libraries**

In [ ]:
from datetime import datetime, timedelta
import numpy as np
from os import path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import json

from astropy.table import Table
from astropy.io import fits
from astropy.wcs import WCS

**Satphotometry library**<br>
satphotometry.satorbit must be imported after starting up SPICE kernel

In [ ]:
from satphotometry import gettle,fitsparser

**Observatory setting**

In [ ]:
from input.obs_site.KUPT import *
# obs_gd_lon_deg = 130.0 + (12.0 + 42.0 / 60.0) / 60.0    # [deg]
# obs_gd_lat_deg = 33.0 + (35.0 + 56.0 / 60.0) / 60.0     # [deg]
# obs_gd_height  = 0.073  # [km]

# wavelength_m = 0.5e-6   #[m]
# aperture_m   = 508.0e-3 #[m]

obs_params = [obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,wavelength_m,aperture_m]

**Space-track.org settings**<br>
Set your space-track.org identity and password at `./config/space-track-org.config`

In [ ]:
with open(f'{base_PATH}config/space-track-org.config') as f:
    st_config = json.load(f)
    st_user_id  = st_config["identity"] # space-track.org user id | str
    st_password = st_config["password"] # space-track.org password | str

### FITS Handling
**Read FITS file**<br>
This cell should be re-written for FITS products from each telescope / observatory

In [ ]:
hdu = fits.open(image_PATH)

ccd_header = hdu[fits_ccd_hdu].header
ccd_data = hdu[fits_ccd_hdu].data

if fits_primary_hdu is not None:
    main_header = hdu[fits_primary_hdu].header
else:
    main_header = ccd_header

ccd_wcs = WCS(ccd_header)

**Observation date, time and exposure**

In [ ]:
metadata = fitsparser.LEOfitsparser.parse_metadata(main_header,ccd_header,[],[],True,False)
# matadata = fitsparser.ASTR499.integrate_ALEX_metadata(metadata,row)

if obs_metadata == "AUTO":
    obs_exposure = metadata["EXPTIME"]
    obs_begin = metadata["DATE-OBS"]
    if len(obs_begin)<19:
        obs_begin = obs_begin + "T" + ccd_header["UT"]

obs_step = obs_exposure/4
obs_begin_dt = datetime.strptime(obs_begin[0:19], "%Y-%m-%dT%H:%M:%S")
obs_end = datetime.strftime(obs_begin_dt+timedelta(seconds=obs_exposure+obs_step), "%Y-%m-%dT%H:%M:%S")

**RA-DEC Range**<br>
Calculate RA/DEC range of searching

In [ ]:
def ra_range(radec_values):
    ra_values = radec_values[:, 0]
    dec_values = radec_values[:, 1]

    if ra_values.max() - ra_values.min() < 180:
        return (
            ra_values.min(),   # ra_min_1
            ra_values.max(),   # ra_max_1
            None,              # ra_min_2
            None,              # ra_max_2
            dec_values.min(),
            dec_values.max()
        )
    else:
        high_cluster = ra_values[ra_values > 180]
        low_cluster  = ra_values[ra_values <= 180]

        ra_min_1 = high_cluster.min()
        ra_max_1 = 360

        ra_min_2 = 0
        ra_max_2 = low_cluster.max()
        return (
            ra_min_1,
            ra_max_1,
            ra_min_2,
            ra_max_2,
            dec_values.min(),
            dec_values.max()
        )

naxis1 = len(ccd_data[0])
naxis2 = len(ccd_data)

x_min = -naxis1*(search_range/2-0.5)
x_max = naxis1*(search_range/2+0.5)
y_min = -naxis2*(search_range/2-0.5)
y_max = naxis2*(search_range/2+0.5)

xy = np.array([[x_min, y_min],
               [x_max, y_min],
               [y_min, y_max],
               [x_max,y_max]])

corner_radec = ccd_wcs.all_pix2world(xy,1)
ra_min_1, ra_max_1, ra_min_2, ra_max_2, dec_min, dec_max = ra_range(corner_radec)

### Orbit calculation

**Download TLE file**<br>
Download latest Two-Line Element set from space-track.org

In [ ]:
tle_PATH = "{0}/tle/{1}.tle".format(input_PATH,obs_begin[0:10])
if path.exists(tle_PATH) is not True:
    status,tle_result = gettle.space_track.get_past_TLEs(obs_begin[0:10],tle_dt_range,st_user_id,st_password)
    if status == 200:
        with open(tle_PATH,mode="w") as f:
            f.write(tle_result)
    else:
        raise ValueError("Failed to download Two-Line Element set or NORAD catalog number is invalid.")

**Parse TLE file**

In [ ]:
tle_dict = gettle.parse.parse_tles_file(tle_PATH)
filtered_tle_dict = gettle.parse.filter_nearest_tles(tle_dict, obs_begin[0:19])

**SGP4 Propagation**

In [ ]:
import cal_satorbit

# output list
possible_objs = []

# output display
print("--------------------------------------------------------------------------------")
print(" Search range")
print("--------------------------------------------------------------------------------")
print(" Right Ascension [deg] : {0} < RA < {1}".format(round(ra_min_1,4),round(ra_max_1,4)) if ra_min_2 is None else " Right Ascension : {0} <= RA < {1} OR {2} < RA <= {3}".format(round(ra_min_2,4),round(ra_max_2,4),round(ra_min_1,4),round(ra_max_1,4)))
print(" Declination     [deg] : {0} < DEC < {1}".format(round(dec_min,4),round(dec_max,4)))
print(" Search objects type   : ALL SATELLITES AND DEBRIS")
print(" Search date range     : {0} +- {1} DAYS".format(obs_begin[0:10],tle_dt_range))
print("--------------------------------------------------------------------------------")
print(" Artificial objects in search range")
print("--------------------------------------------------------------------------------")

# SGP4 Propagation to all objects
intldess = []
for i in tqdm(range(0,len(filtered_tle_dict)),desc="Calculating SGP4 Propagation "):
    try:
        objname,norad_id,intldes,output = cal_satorbit.cal_satorbit(obs_begin,obs_end,obs_step,[filtered_tle_dict[i]["name"],filtered_tle_dict[i]["line1"],filtered_tle_dict[i]["line2"]],obs_params,spice_myfile_PATH)
    except:
        continue

    # filter out objects in umbra
    output = output[output["umbra"] == False]

    output_ra = output["ra[deg]"]
    output_dec = output["dec[deg]"]

    for j in range(0,len(output)-1):
        main_ra = output[j]["ra[deg]"]
        next_ra = output[j+1]["ra[deg]"]
        main_dec = output[j]["dec[deg]"]
        next_dec = output[j]["dec[deg]"]

        if abs(main_ra-next_ra) > 180:
            if main_ra > 180:
                for k in range(0,99):
                    output_ra = np.append(output_ra,(main_ra + k*(next_ra + 360 - main_ra)/100)%360)
                    output_dec = np.append(output_dec,main_dec + k*(next_dec - main_dec)/100)
            else:
                for k in range(0,99):
                    output_ra = np.append(output_ra,(main_ra + k*(next_ra - 360  - main_ra)/100)%360)
                    output_dec = np.append(output_dec,main_dec + k*(next_dec - main_dec)/100)
        else:
            for k in range(0,99):
                output_ra = np.append(output_ra,main_ra + k*(next_ra - main_ra)/100)
                output_dec = np.append(output_dec,main_dec + k*(next_dec - main_dec)/100)

    # T/F search range
    ra_tf_1 = ra_min_1 < output_ra
    ra_tf_2 = output_ra < ra_max_1
    dec_tf_1 = dec_min < output_dec
    dec_tf_2 = output_dec < dec_max
    if ra_min_2 is not None:
        ra_tf_3 = ra_min_2 < output_ra
        ra_tf_4 = output_ra < ra_max_2
        radec_tf_1 = [True if ((ra_tf_1[i] is np.True_ and ra_tf_2[i] is np.True_) and (dec_tf_1[i] is np.True_ and dec_tf_2[i] is np.True_)) else False for i in range(0,len(output_ra))]
        radec_tf_2 = [True if ((ra_tf_3[i] is np.True_ and ra_tf_4[i] is np.True_) and (dec_tf_1[i] is np.True_ and dec_tf_2[i] is np.True_)) else False for i in range(0,len(output_ra))]
        possible_obj = any([any(radec_tf_1),any(radec_tf_2)])
    else:
        radec_tf = [True if ((ra_tf_1[i] is np.True_ and ra_tf_2[i] is np.True_) and (dec_tf_1[i] is np.True_ and dec_tf_2[i] is np.True_)) else False for i in range(0,len(output_dec))]
        possible_obj = any(radec_tf)

    # save to and display possible object
    if possible_obj:
        possible_objs.append([objname,norad_id,intldes,output])
        if intldes not in intldess:
            tqdm.write(" * {0} (INTLDES {1} / NORAD CAT ID {2})".format(objname,intldes,norad_id))
        intldess.append(intldes)
print("--------------------------------------------------------------------------------")

### Output
**Plot**

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(ccd_data, cmap='gray', origin='lower', vmin=np.percentile(ccd_data, 5), vmax=np.percentile(ccd_data, 95))
intldess = []
for object in possible_objs:
    orbit = object[3]
    orbit_radecs = np.array([[f["ra[deg]"],f["dec[deg]"]] for f in orbit if f["umbra"]==False])
    orbit_xy = ccd_wcs.wcs_world2pix(orbit_radecs,1)
    orbit_x = [f[0] for f in orbit_xy]
    orbit_y = [f[1] for f in orbit_xy]
    plt.plot(orbit_x,orbit_y,label="{0} (INTLDES {1} / NORAD CAT ID {2})".format(object[0],object[2],object[1]),lw=3)
    plt.scatter(orbit_x,orbit_y)
    if object[2] not in intldess:
        plt.annotate("  {0} {1}".format(object[0],object[2]),(orbit_x[0], orbit_y[0]),fontsize=20)
    intldess.append(object[2])

plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)
# plt.xlim(0,naxis1)
# plt.ylim(0,naxis2)
plt.legend(bbox_to_anchor=(0.5, -0.25),loc='lower center',ncol=3)
plt.show()

**SPICE toolkit kernel clear**

In [ ]:
spice.kclear()